In [4]:
import random
import polars as pl
from pathlib import Path
import json

In [5]:
nlg_ds = pl.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv')#,  n_rows=1000)
nlg_ds = nlg_ds.with_columns(pl.col("ingredients").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.col("directions").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.arange(pl.len()).alias("index"))

In [11]:
import random
random.seed(123)
eval_data_idx = random.sample(range(0,nlg_ds.shape[0]), k=1000)

In [12]:
out_ds = nlg_ds.filter(pl.col("index").is_in(eval_data_idx))

In [18]:
# Don't accidentally overwrite
#with open("../../data/eval_data/eval_data.json", "w") as f:
    #json.dump(out_ds.select('index', 'title', 'ingredients').to_dicts(), f, indent=2)

In [34]:
with open("../../data/eval_data/processed_data.json", "r") as f:
    processed_ds = pl.read_json(f)

In [36]:
processed_ds.head()

index,title,ingredients,google_search_query,main_food_items_and_preparations,important_ingredients_for_search,google_search_query_processed
i64,str,list[str],str,list[str],list[str],struct[2]
1461,"""Apricot Cake""","[""2 c. sugar"", ""4 eggs"", … ""1/2 c. chopped nuts (English or pecans)""]","""apricot cake recipe with baby …","[""sugar"", ""eggs"", … ""chopped nuts (English or pecans)""]","[""apricot baby food"", ""flour"", … ""Wesson oil""]","{""Apricot Cake"",[""baby food"", ""oil"", … ""cinnamon""]}"
1490,"""Angel Delights""","[""1 c. miniature marshmallows"", ""1 c. broken pretzels"", … ""1 lb. almond bark""]","""angel delights recipe with mar…","[""miniature marshmallows"", ""broken pretzels"", … ""almond bark""]","[""marshmallows"", ""pretzels"", … ""almond bark""]","{""Angel Delights"",[""marshmallows"", ""pretzels"", … ""almond bark""]}"
3374,"""Savory Sausage Casserole""","[""1 lb. bulk pork sausage"", ""1 c. uncooked rice"", … ""1/2 c. toasted halved or slivered blanched almonds""]","""sausage and rice casserole wit…","[""bulk pork sausage"", ""uncooked rice"", … ""toasted halved or slivered blanched almonds""]","[""pork sausage"", ""rice"", … ""almonds""]","{""Savory Sausage Casserole"",[""rice"", ""chicken noodle soup mix"", ""almonds""]}"
7006,"""Zemakove Knedliky A Kysele Zel…","[""2 c. mashed potatoes"", ""1/2 tsp. salt"", … ""1 1/2 c. flour""]","""potato dumplings with sauerkra…","[""mashed potatoes"", ""salt"", … ""flour""]","[""mashed potatoes"", ""sauerkraut"", … ""egg""]","{""Zemakove Knedliky A Kysele Zeli(Potato Dumplings With Sauerkraut) "",[""sauerkraut"", ""mashed potatoes""]}"
9957,"""Scalloped Sweet Potatoes""","[""1/4 c. sugar"", ""1/2 tsp. cinnamon"", … ""3 Tbsp. margarine""]","""scalloped sweet potatoes and a…","[""sugar"", ""cinnamon"", … ""margarine""]","[""sweet potatoes"", ""apples"", ""cinnamon""]","{""Scalloped Sweet Potatoes"",[""sweet potatoes"", ""apples""]}"


In [37]:
human_eval_data_idx = random.sample(eval_data_idx, k=50)

In [39]:
human_eval_ds = processed_ds.filter(pl.col("index").is_in(human_eval_data_idx))

In [41]:
human_eval_ds.head()

index,title,ingredients,google_search_query,main_food_items_and_preparations,important_ingredients_for_search,google_search_query_processed
i64,str,list[str],str,list[str],list[str],struct[2]
25000,"""Pineapple Cream Cake""","[""1 pkg. white cake mix"", ""2 small pkg. vanilla instant pudding mix"", … ""4 oz. cream cheese""]","""pineapple cream cake with cake…","[""white cake mix"", ""vanilla instant pudding mix"", … ""cream cheese""]","[""white cake mix"", ""vanilla pudding mix"", … ""Dream Whip""]","{""Pineapple Cream Cake"",[""cake mix"", ""vanilla pudding"", … ""dream whip""]}"
135438,"""Pineapple Casserole""","[""2 large cans pineapple tidbits, drained"", ""3/4 c. flour"", … ""1 stick melted butter""]","""pineapple casserole with chedd…","[""pineapple tidbits (drained)"", ""flour"", … ""melted butter""]","[""pineapple"", ""Cheddar cheese"", ""Ritz crackers""]","{""Pineapple Casserole"",[""cheddar cheese"", ""ritz crackers""]}"
230315,"""Low-Fat Chili""","[""1 1/2 lb. ground turkey"", ""1 1/2 c. chopped onion"", … ""3 c. brown rice, cooked""]","""low fat turkey chili with blac…","[""ground turkey"", ""onion (chopped)"", … ""brown rice (cooked)""]","[""ground turkey"", ""black beans"", … ""chili powder""]","{""Low-Fat Chili"",[""black beans"", ""zucchini"", ""brown rice""]}"
272288,"""Almond Bars""","[""1 c. butter"", ""2 c. flour"", ""1/2 c. powdered sugar""]","""3 ingredient almond shortbread…","[""butter"", ""flour"", ""powdered sugar""]","[""butter"", ""flour"", ""powdered sugar""]","{""Almond Bars"",[]}"
275407,"""Pineapple Icebox Cake""","[""1 box yellow cake mix"", ""1 (20 oz.) can crushed pineapple (with juice)"", … ""1 c. chopped pecans""]","""pineapple icebox cake with yel…","[""yellow cake mix"", ""crushed pineapple (with juice)"", … ""chopped pecans""]","[""yellow cake mix"", ""crushed pineapple"", … ""whipped topping""]","{""Pineapple Icebox Cake"",[""yellow cake mix"", ""vanilla pudding""]}"


In [42]:
#with open("../../data/eval_data/human_data.json", "w") as f:
#    json.dump(human_eval_ds.select('index', 'title', 'ingredients').to_dicts(), f, indent=2)